## 1. read data from mongoDB

In [1]:
import pymongo
import re

In [2]:
def mongo_connect_resume(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") # 跟mongodb建立連

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    col = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立
    
def resume_find():
    # 尋找多筆資料
    return col.find({},{"_id":0})

In [3]:
mongo_connect_build('cakeresume', 'Resume') # connect to mongoDB to get job data
job_data = data_find().limit(10)
for i in job_data:
    print(i)
    print('======================================')

{'id': 'wangsagacious', 'value': '\n\n\n\n\n\n\n              王士榮 Sloan Wang\n             \n\n\n\n\n\n                \uf09a\n               \n\n\n\n                \uf0e1\n               \n\n\n\n                \uf099\n               \n\n\n\n                \uf09b\n               \n\n\n\n\n\n\n               產品總監/遊戲製作人/機率工程師\xa0\xa0•\xa0\xa0Taipei Special Municipality，TW\xa0\xa0•\n               \n\n                 [email\xa0protected]\n                \n\n\n\n               您好，我是王士榮，有十多以上遊戲開發經驗，擔任過遊戲製作人、研發主管、企劃主管、敏捷專案負責人、機率工程師等職務。\n               \n\n               擅長從零到有撰寫企劃文案，搭配敏專案捷開發將產品完成並上線，累積經歷有：團隊建置、專案管理、溝通協調、敏捷開發、進度控管、產品調研、需求整合、可行性評估、原型製作、遊戲文件、流程規劃、介面體驗、數值機率、音樂音效、測試驗證、產品上線、數據分析、更新優化…等。\n               \n\n               開發過的遊戲有：棋牌(牛牛、三公、牌九、麻將、鬥地主…)、百人(龍虎、紅黑、骰寶、百家樂、奔馳跑馬…)、捕魚機、老虎機、彈珠、賓果、推幣、娃娃機、籃球、棒球、保齡球、攀岩、賽車、摩托車等遊戲在街機、電腦、網頁、手機等不同平台，大小專案數數十款以上。\n              \n\n\n\n\n\n\n\n\n\n\n                數值機率 MATHEMATICIAN\n               \n\n\n\n\n\n\n\n\n\n\n                完成過推幣、捕魚

## 2. Clean job Description data

### 定義資料清洗function

In [4]:
def isEnglish(s): # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def isNumber(s): # 檢查字元是否為數字
    return s.isdigit()


def string_clean(CVdescription):  # 字串清洗
    '''
    input: string (original CV_description)
    output: string (clean CV_description)
    '''
    cv_desc = CVdescription.split('\r\n') # 根據換行符號轉乘 List格式
    cv_words = ''
    
    for words in cv_desc:
        words = re.sub(r'[^\w\s]',' ', words)  # remove all punctuations
        words = re.sub(r'\d+', ' ', words)  # remove all numbers
        words = words.strip() # remove white space
        cv_words += words
    
    return (cv_words)


## 3. 使用 Jieba 分詞

In [5]:
import jieba

In [6]:
# 指定辭典檔
jieba.load_userdict('./Jobcontent_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.766 seconds.
Prefix dict has been built successfully.


In [7]:
# 指定 Stop words檔案

with open(file='./Jobcontent_stopwords.txt',mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]

In [8]:
# 使用結巴斷詞，產生 list of words

def jieba_cut(data):
    
    # 使用結巴斷詞
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []
    
    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  #排除空值
            continue
        elif i in stop_words: # 排除stopwords
            continue
        elif isEnglish(i) == False and len(i) == 1: #排除單一中文字
            continue
        elif isNumber(i) == True: #排除數字
            continue
        else:
            lst_seg.append(i)
   
    return lst_seg

In [9]:
mongo_connect_build('cakeresume', 'Resume')

In [10]:
job_data = data_find().limit(10)

In [11]:
for i in job_data:
    resume_clean = string_clean(i['value'])
    resume_result = jieba_cut(resume_clean)
    print(resume_result)
    print('============================================')

['王士榮', 'Sloan', 'Wang', '產品', '遊戲', '工程師', 'Taipei', 'Special', 'Municipality', 'TW', 'protected', '王士榮', '十多', '遊戲', '開發', '遊戲', '研發', '主管', '企劃', '主管', '敏捷', '專案', '工程師', '職務', '擅長', '零到', '撰寫', '企劃', '文案', '專案', '開發', '產品', '經歷', '建置', '專案管理', '協調', '敏捷', '開發', '進度', '控管', '產品', '調研', '整合', '評估', '遊戲', '文件', '流程', '規劃', '音樂', '音效', '測試', '驗證', '產品', '數據', '分析', '優化', '開發', '遊戲', '棋牌', '牛牛', '牌九', '麻將', '龍虎', '紅黑', '骰寶', '奔馳', '捕魚機', '老虎', '彈珠', '賓果', '推幣', '籃球', '棒球', '保齡球', '攀岩', '賽車', '車等', '遊戲', '街機', '電腦', '網頁', '手機', '平台', '專案', '數數', '十款', 'MATHEMATICIAN', '過推幣', '捕魚', '彈珠', '遊戲', '專案', '計算', '超過數', '十款', '老虎', '專案', '研發', '遊戲', '設計', '數學', '模型', '建立', 'EXCEL', 'VBA', '撰寫', '程式', '巨集', '模擬', '老虎', '遊戲', '數據', '產出', '報表', '繪製', '出吐', '波動', '用戶', '用戶', 'UI', 'UX', '用戶', '觀察', '觀察', '遊戲', '設計', '框圖', 'Wireframe', 'Mock', '作業', '完善', '字型', '顏色', '工具', 'Axure', 'RP', 'Invision', 'Studio', 'Balsamiq', 'Mock', '敏捷', '開發', 'SCRUM', 'DEVELOPMENT', '敏捷', '教練', 'Master', '敏捷', 'Owner', 

['邱修德', 'Shoulder', 'Chu', '行銷企劃', '專案', '專案', '挑戰', '方案', '增添', '色彩', '職務', '擔任', '橋梁', '勾勒', '藍圖', '實景能', '精準', 'protected', '專案管理', '商業', '專案', '企劃', '規劃', '企業', '行銷', '專案', '研討會', '專家', '講座', '成果', '發表會', '網站', '規劃', '企業', '影片', '拍攝', '線上', '內部', '導入', 'E化', '系統', 'Jun', 'Jan', '檢視', '系統', '產出', '報表', '檢視', '專案', '狀態', '資源', '共享', '建立', '客戶', '廠商', '資訊', '資料庫', '無紙化', '專案', '補修', '會議', '預約', '擅長', '工具', 'Excel', '狀況', '調查', '樞紐', '分析', 'SurveyCake', 'P', 'PT', '提案', '企劃', '簡報', 'Keep', '專案', '代辦', 'Evernote', '廠商', '知識', '資訊管理', 'Google', 'A', 'ds', '數位', '認證', '公益', '大學', '海外', '志工', 'NEC', '台灣', '行銷', '主任', 'May', '品牌', '行銷', '傳播', '策略', '規劃', '專案管理', '正平', '整合', '行銷', '資深', '專案', 'Oct', 'Jan', '掌上', '百場', '專案', '研討會', '工作坊', '攤位', '展示', '校園', '徵才', '講座', '成果', '發表會', '網站', '規劃', '影片', '拍攝', '專案', '親子', '千人', '線上', '專案', '預算', '義務役', '總統府', '憲兵', '退役', 'Sep', 'Sep', '義務役', '將軍旅', '撰成單', '篇文章', 'Facebook', '該篇', '文獲', '勤益', '科技', '大學', '電子', '工程系', 'Sep', 'Jun', '國際', '志工', '社長', 

['Guo', 'Cian', 'Yu', 'UI', 'UX', 'Designer', 'am', 'multifaceted', 'designer', 'passionate', 'branding', 'design', 'UI', 'UX', 'am', 'active', 'learner', 'keen', 'expand', 'knowledge', 'My', 'point', 'view', 'design', 'esign', 'solve', 'problems', 'social', 'problems', 'environment', 'better', 'just', 'looks', 'beautiful', 'Design', 'Portfolio', 'LinkedIn', 'protected', 'Skills', 'UX', 'Information', 'analysis', 'flows', 'Wireframes', 'Usability', 'testing', 'UI', 'Defining', 'design', 'systems', 'Design', 'ing', 'branding', 'product', 'Design', 'ing', 'mock', 'interactive', 'prototypes', 'Coding', 'HTML', 'CSS', 'Design', 'Tools', 'Adobe', 'Figma', 'Sketch', 'Zeplin', 'Axure', 'Illustrator', 'Photoshop', 'Soft', 'Skills', 'Ability', 'quickly', 'adapt', 'environment', 'ideas', 'communication', 'skills', 'collaborate', 'stakeholders', 'A', 'starter', 'Excel', 'skills', 'time', 'management', 'skills', 'deliver', 'task', 'time', 'quality', 'Work', 'Experience', 'UI', 'UX', 'Designer', 'G